In [11]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import keras
from keras import backend as K

In [10]:
TF_ENABLE_ONEDNN_OPTS=0

In [2]:
BACKBONE='resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [12]:
def jacard_coef(y_true, y_pred):
    # y_true=y_true.astype(np.uint8)
    # y_prede=y_pred.astype(np.uint8)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0))


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

In [37]:
def binary_tangent(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    tangent=tf.tan((y_true-y_pred))
    return (K.sum(tangent)/float(len(tangent)))

def binary_tangent_loss(y_true, y_pred):
    return -binary_tangent(y_true, y_pred)

In [5]:
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.5, 0.5])) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [6]:
SIZE_X = 256
SIZE_Y = 256

In [20]:
train_images = []
train_labels=[]
for directory_path in (glob.glob("SBU-shadow/SBUTrain4KRecoveredSmall/ShadowImages")):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
#         print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
#Convert list to array for machine learning processing        
train_images = np.array(train_images)

NameError: name 'label' is not defined

In [ ]:
train_masks = [] 
for directory_path in sorted(glob.glob("SBU-shadow/SBUTrain4KRecoveredSmall/ShadowMasks/")):
    for mask_path in sorted(glob.glob(os.path.join(directory_path, "*.png"))):
        print(mask_path)
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
        #mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        train_masks.append(mask)
        train_labels.append(label)
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

In [14]:
X=np.load('imgs.npy')
Y=np.load('masks.npy')
Y = np.expand_dims(Y, axis=3)
Y=(Y>0.5).astype(np.float32)

In [15]:
# X=train_images
# Y=train_masks
# Y = np.expand_dims(Y, axis=3)

In [16]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [17]:
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

In [38]:
model = sm.Unet(BACKBONE,(256,256,3),classes=1,activation='sigmoid', encoder_weights='imagenet')
model.compile(optimizer= 'adam', loss=[binary_tangent_loss],
              metrics = [binary_tangent])
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 256, 256, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d_34 (ZeroPadding  (None, 262, 262, 3)  0          ['bn_data[0][0]']                
 2D)                                                                                              
                                                                                            

 zero_padding2d_40 (ZeroPadding  (None, 66, 66, 64)  0           ['stage1_unit3_relu1[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage1_unit3_conv1 (Conv2D)    (None, 64, 64, 64)   36864       ['zero_padding2d_40[0][0]']      
                                                                                                  
 stage1_unit3_bn2 (BatchNormali  (None, 64, 64, 64)  256         ['stage1_unit3_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage1_unit3_relu2 (Activation  (None, 64, 64, 64)  0           ['stage1_unit3_bn2[0][0]']       
 )                                                                                                
          

 zation)                                                                                          
                                                                                                  
 stage2_unit3_relu2 (Activation  (None, 32, 32, 128)  0          ['stage2_unit3_bn2[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padding2d_47 (ZeroPadding  (None, 34, 34, 128)  0          ['stage2_unit3_relu2[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage2_unit3_conv2 (Conv2D)    (None, 32, 32, 128)  147456      ['zero_padding2d_47[0][0]']      
                                                                                                  
 add_21 (A

 2D)                                                                                              
                                                                                                  
 stage3_unit2_conv2 (Conv2D)    (None, 16, 16, 256)  589824      ['zero_padding2d_53[0][0]']      
                                                                                                  
 add_24 (Add)                   (None, 16, 16, 256)  0           ['stage3_unit2_conv2[0][0]',     
                                                                  'add_23[0][0]']                 
                                                                                                  
 stage3_unit3_bn1 (BatchNormali  (None, 16, 16, 256)  1024       ['add_24[0][0]']                 
 zation)                                                                                          
                                                                                                  
 stage3_un

 zation)                                                                                          
                                                                                                  
 stage3_unit6_relu1 (Activation  (None, 16, 16, 256)  0          ['stage3_unit6_bn1[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padding2d_60 (ZeroPadding  (None, 18, 18, 256)  0          ['stage3_unit6_relu1[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage3_unit6_conv1 (Conv2D)    (None, 16, 16, 256)  589824      ['zero_padding2d_60[0][0]']      
                                                                                                  
 stage3_un

 2D)                                                                                              
                                                                                                  
 stage4_unit3_conv1 (Conv2D)    (None, 8, 8, 512)    2359296     ['zero_padding2d_66[0][0]']      
                                                                                                  
 stage4_unit3_bn2 (BatchNormali  (None, 8, 8, 512)   2048        ['stage4_unit3_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage4_unit3_relu2 (Activation  (None, 8, 8, 512)   0           ['stage4_unit3_bn2[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padd

 decoder_stage2b_conv (Conv2D)  (None, 64, 64, 64)   36864       ['decoder_stage2a_relu[0][0]']   
                                                                                                  
 decoder_stage2b_bn (BatchNorma  (None, 64, 64, 64)  256         ['decoder_stage2b_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 decoder_stage2b_relu (Activati  (None, 64, 64, 64)  0           ['decoder_stage2b_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 decoder_stage3_upsampling (UpS  (None, 128, 128, 64  0          ['decoder_stage2b_relu[0][0]']   
 ampling2D)                     )                                                                 
          

In [39]:
history=model.fit(x_train, 
          y_train,
          batch_size=16, 
          epochs=10,
          verbose=1,
          validation_data=(x_val, y_val))

Epoch 1/10
  8/205 [>.............................] - ETA: 6:19 - loss: 18291.7910 - binary_tangent: -18291.7910

KeyboardInterrupt: 